<p style="font-weight:bold;"> <span style="font-size: 36px"> Extention Methods </span> </p

In [0]:
#!import "../DataModel/DataStructure"
#!import "ApplicationMessage"

# Loading external dependencies

In [0]:
#r "nuget:morelinq"

In [0]:
using static MoreLinq.Extensions.ZipLongestExtension;

# Enumerable Extensions

In [0]:
static T GetValidElement<T>(this ICollection<T> collection, int index)
{   
    var count = collection.Count;
    if (collection == null || count == 0)
        return default(T);

    if (index < 0)
    {
        ApplicationMessage.Log(Error.NegativeIndex);
        return default;
    }

    return index < count
                ? collection.ElementAt(index)
                : collection.ElementAt(count -1);
}

In [0]:
public static Dictionary<TKey, TResult> ToDictionaryGrouped<TSource, TKey, TResult>(this IEnumerable<TSource> source, Func<TSource, TKey> keySelector, Func<IGrouping<TKey, TSource>, TResult> elementSelector) => source.GroupBy(keySelector).ToDictionary(g => g.Key, elementSelector);

In [0]:
public static IDataCube<TTarget> SelectToDataCube<TSource, TTarget>(this IEnumerable<TSource> source, Func<TSource, bool> whereClause, Func<TSource, TTarget> selector) => source.Where(whereClause).Select(selector).ToDataCube();

In [0]:
public static IDataCube<TTarget> SelectToDataCube<TSource, TTarget>(this IEnumerable<TSource> source, Func<TSource, TTarget> selector) => source.SelectToDataCube(x => true, selector);

In [0]:
public static double[] Prune(this IEnumerable<double> source, double precision = Precision) => source.Reverse().SkipWhile(x => Math.Abs(x) < precision).Reverse().ToArray();

In [0]:
public static double[] AggregateDoubleArray(this IEnumerable<IEnumerable<double>> source) => source.Where(x => x is not null).DefaultIfEmpty(Enumerable.Empty<double>()).Aggregate((x, y) => x.ZipLongest(y, (a, b) => a + b)).ToArray();

# String Extensions

In [0]:
using System.Globalization;

In [0]:
public static double CheckStringForExponentialAndConvertToDouble (this string s)
{   
    if (s == null) return default;
    if (double.TryParse(s, NumberStyles.Number, CultureInfo.InvariantCulture, out var doubleValue)) return doubleValue;
    else { ApplicationMessage.Log(Error.ParsingScientificNotation, s); return 1; }
}

# Enum Extentions

In [0]:
public static bool Contains<T>(this T value, T lookingForFlag) 
    where T : struct
{
    int intValue = (int) (object) value;
    int intLookingForFlag = (int) (object) lookingForFlag;
    return ((intValue & intLookingForFlag) == intLookingForFlag);
}

# ToIdentityString

In [0]:
using System.Text;
public static string ToIdentityString<T>(this T v, params string[] excludingProperties) where T : class
{
   StringBuilder sb = new StringBuilder();
   var propertyInfos = v.GetType()
                        .GetProperties()
                        .Where(x => Attribute.IsDefined(x, typeof(IdentityPropertyAttribute)) && !excludingProperties.Contains(x.Name))
                        .OrderByDescending(x => x.PropertyType.Name).ThenByDescending(x => x.Name)
                        .Select(x => sb.Append(x.Name).Append(":").Append(v.GetType().GetProperty(x.Name)?.GetValue(v, null)).Append(", ")).ToArray();
   return propertyInfos.Count() == 0? v.ToString() : propertyInfos.Select(p => p.ToString()).ToArray().Last();
}

# ToString with including properties

In [0]:
public static string ToStringWith<T>(this T variable, string[] properties) where T : BaseVariableIdentity
{
    var propertiesRead = variable.ToString().Split('{', '}')[1].Split(',');
    var propertiesFiltered = propertiesRead.Where(x=>  properties.Contains(x.Split('=')[0].Trim()));
    return string.Join(", ", propertiesFiltered).Trim();
}